<a href="https://colab.research.google.com/github/6aitam9/train/blob/6aitam9-patch-1/%D0%A4%D0%B8%D0%BD%D0%B0%D0%BB%D1%8C%D0%BD%D0%BE%D0%B5_%D0%B7%D0%B0%D0%B4%D0%B0%D0%BD%D0%B8%D0%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Итоговый проект. Улицы. Объекты культурного наследия.

In [115]:
# TODO собрать установку всех необходимых модулей в одном месте
!pip install geopandas
!pip install osmnx
!pip install mapclassify

In [116]:
# TODO собрать импорты всех модулей в одном месте
import pandas as pd
import geopandas as gpd
import osmnx as ox

In [117]:
TILES = "CartoDB positron"  # Название подложки для карт

# TODO указать любой район Санкт-Петербург из OSM https://wiki.openstreetmap.org/wiki/RU:%D0%A1%D0%B0%D0%BD%D0%BA%D1%82-%D0%9F%D0%B5%D1%82%D0%B5%D1%80%D0%B1%D1%83%D1%80%D0%B3/%D0%A0%D0%B0%D0%B9%D0%BE%D0%BD%D1%8B
TERRITORY_NAME = ['Василеостровский район, Санкт-Петербург']  # название территории для которой будут строиться слои

# TODO указать ссылку на файл из вашего github репозитория, которая начинается с https://raw.githubusercontent.com/
KGIOP_FILE_URL = 'https://raw.githubusercontent.com/6aitam9/train/main/geojson_layers/kgiop_objects.geojson'  # ссылка на слой с объектами культурного наследия
STREETS_FILE_URL = 'https://raw.githubusercontent.com/6aitam9/train/main/geojson_layers/streets.geojson'  # ссылка на слой с улицами

## Территория

### Загрузка территории из OSM (Extract)

In [118]:
district = ox.geocode_to_gdf(TERRITORY_NAME)# TODO загрузить geodataframe с геометрией для территории TERRITORY_NAME
district

,geometry,bbox_north,bbox_south,bbox_east,bbox_west,place_id,osm_type,osm_id,lat,lon,class,type,place_rank,importance,addresstype,name,display_name
0,"POLYGON ((30.17787 59.94437, 30.17790 59.94403...",59.964376,59.91726,30.316861,30.177873,181717520,relation,1114252,59.939275,30.243876,boundary,administrative,10,0.400226,administrative,Vasileostrovsky District,"Vasileostrovsky District, округ Гавань, Saint ..."


## Улицы

### Загрузка файла с улицами (Extract)

In [119]:
# TODO отфильтровать улицы по маске геометрии территории полученной ранее
gdf = gpd.read_file(STREETS_FILE_URL, mask=district)
gdf

,name,geometry
0,Кожевенная линия,"LINESTRING (3366312.890 8383418.064, 3366318.9..."
1,улица Нахимова,"LINESTRING (3364793.702 8386752.080, 3364743.2..."
2,16-17-я линии В.О.,"LINESTRING (3368203.462 8387924.925, 3368216.1..."
3,Клубный переулок,"LINESTRING (3368114.551 8385036.156, 3368124.6..."
4,переулок Декабристов,"LINESTRING (3367355.174 8388720.917, 3367360.5..."
...,...,...
821,бульвар Александра Грина,"LINESTRING (3362469.039 8385708.171, 3362340.7..."
822,набережная реки Смоленки,"LINESTRING (3369235.249 8388117.669, 3369199.1..."
823,9-я линия В.О.,"LINESTRING (3369481.065 8388327.601, 3369496.5..."
824,Большой проспект В.О.,"LINESTRING (3368901.691 8384926.614, 3368882.4..."


### Обработка данных с улицами (Transform)

In [120]:
# TODO сгруппировать и объединить геометрии с одинаковыми названиями
group_gdf = gdf.dissolve(by='name')

### Сохранение слоя с улицами (Load)

In [121]:
# TODO переименовать столбцы в русские названия, кроме столбца geometry
rename_columns = {
    "name": "Название"
}

rename_gdf = group_gdf.rename(columns=rename_columns)
# TODO для того чтобы переименовать индекс, нужно обратиться и нему и вызвать от него метод rename (https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Index.rename.html)

In [122]:
# TODO сохранить слой в географической проекции в формате GeoJSON
rename_gdf.to_crs(4326).to_file('streat.geojson', driver='GeoJSON')

## Объекты культурного наследия

### Загрузка объектов культурного наследия (Extract)

In [123]:
# TODO отфильтровать улицы по маске геометрии территории полученной ранее
gdf = gpd.read_file(KGIOP_FILE_URL, mask=district)
gdf

,id,ensemble_name,object_name,occurrence_time,object_location,historical_category,normative_act,object_type,geometry
0,1260,—,Дом А.Р. Гешвенда,1880-1881,"9-я линия В.О., 32",выявленный объект культурного наследия,Приказ председателя КГИОП № 15 от 20.02.2001,Памятник,POINT (3370436.885 8386572.161)
1,1021,Дом И.В. Голубина (И.И. Зайцевского),Флигель,нач. 19 в.; 1834,"Репина ул., 17",выявленный объект культурного наследия,Приказ председателя КГИОП № 15 от 20.02.2001,Памятник,POINT (3371907.580 8386491.569)
2,1015,—,Дом Г.И. Головкина (А.Н. Чичагова),1720-е - 1730-е; 1878,"1-я линия В.О., 10; Репина ул., 9",выявленный объект культурного наследия,Приказ председателя КГИОП № 15 от 20.02.2001,Памятник,POINT (3372020.006 8386392.023)
3,1016,—,Дом Леонтьева М.И. (Шуберта Ф.Ф.),"1722-1741, 1807-1809","1-я линия В.О., 12",объект культурного наследия федерального значения,Постановление Правительства РФ № 527 от 10.07....,Памятник,POINT (3372009.644 8386417.444)
4,1017,—,Дом А. Сверчкова (Кушелевой),1730-1740-е; 1815; 1863,"1-я линия В.О., 14, лит. А",объект культурного наследия регионального знач...,Распоряжение КГИОП № 139-р от 23.04.2021,Памятник,POINT (3372026.660 8386473.431)
...,...,...,...,...,...,...,...,...,...
777,9441,Комплекс зданий Французского благотворительног...,Ограда,1905,"14-я линия В.О., 57-61",объект культурного наследия регионального знач...,Распоряжение КГИОП от 22.03.2016 № 10-92,Памятник,POINT (3369220.867 8386925.034)
778,9442,Комплекс зданий Французского благотворительног...,Сад,кон. 19 - нач. 20 в.,"14-я линия В.О., 57-61",объект культурного наследия регионального знач...,Распоряжение КГИОП от 22.03.2016 № 10-92,Ансамбль,POINT (3369290.775 8386927.924)
779,9517,—,Главное здание Государственного оптического ин...,1947-1956,"Кадетская линия, 5, корп. 2, лит. В",объект культурного наследия регионального знач...,Распоряжение КГИОП от 01.07.2019 № 365-р,Памятник,POINT (3372240.630 8387172.627)
780,9605,Два жилых флигеля,Жилой флигель,кон. 18 в.; 1848,"4-я линия В.О., 39, лит. Б",объект культурного наследия регионального знач...,Распоряжение КГИОП от 05.08.2019 № 437-р,Памятник,POINT (3370714.885 8387675.618)


### Обработка объектов культурного наследия (Transform)

In [124]:
# TODO добавить два столбца lon и lat, в которых будут долгота и широта
gdf["lon"] = gdf.to_crs(3857).geometry.centroid.to_crs(4326).x
gdf["lat"] = gdf.to_crs(3857).geometry.centroid.to_crs(4326).x

### Сохранение слоя с объектами культурного наследия (Load)

In [125]:
# TODO переименовать столбцы в русские названия, кроме столбца geometry
rename_columns = {
    "ensemble_name": "Название ансамбля",
    "object_name": "Имя объекта",
    "occurrence_time": "Дата создания",
    "object_location": "Локация объекта",
    "historical_category": "Историческая категория",
    "normative_act": "Нормативный акт",
    "object_type": "Тип объекта",
    "lon":"Долгота",
    "lat":"Широта",

}
rename_gdf = gdf.rename(columns=rename_columns)
rename_gdf

,id,Название ансамбля,Имя объекта,Дата создания,Локация объекта,Историческая категория,Нормативный акт,Тип объекта,geometry,Долгота,Широта
0,1260,—,Дом А.Р. Гешвенда,1880-1881,"9-я линия В.О., 32",выявленный объект культурного наследия,Приказ председателя КГИОП № 15 от 20.02.2001,Памятник,POINT (3370436.885 8386572.161),30.277150,30.277150
1,1021,Дом И.В. Голубина (И.И. Зайцевского),Флигель,нач. 19 в.; 1834,"Репина ул., 17",выявленный объект культурного наследия,Приказ председателя КГИОП № 15 от 20.02.2001,Памятник,POINT (3371907.580 8386491.569),30.290361,30.290361
2,1015,—,Дом Г.И. Головкина (А.Н. Чичагова),1720-е - 1730-е; 1878,"1-я линия В.О., 10; Репина ул., 9",выявленный объект культурного наследия,Приказ председателя КГИОП № 15 от 20.02.2001,Памятник,POINT (3372020.006 8386392.023),30.291371,30.291371
3,1016,—,Дом Леонтьева М.И. (Шуберта Ф.Ф.),"1722-1741, 1807-1809","1-я линия В.О., 12",объект культурного наследия федерального значения,Постановление Правительства РФ № 527 от 10.07....,Памятник,POINT (3372009.644 8386417.444),30.291278,30.291278
4,1017,—,Дом А. Сверчкова (Кушелевой),1730-1740-е; 1815; 1863,"1-я линия В.О., 14, лит. А",объект культурного наследия регионального знач...,Распоряжение КГИОП № 139-р от 23.04.2021,Памятник,POINT (3372026.660 8386473.431),30.291431,30.291431
...,...,...,...,...,...,...,...,...,...,...,...
777,9441,Комплекс зданий Французского благотворительног...,Ограда,1905,"14-я линия В.О., 57-61",объект культурного наследия регионального знач...,Распоряжение КГИОП от 22.03.2016 № 10-92,Памятник,POINT (3369220.867 8386925.034),30.266226,30.266226
778,9442,Комплекс зданий Французского благотворительног...,Сад,кон. 19 - нач. 20 в.,"14-я линия В.О., 57-61",объект культурного наследия регионального знач...,Распоряжение КГИОП от 22.03.2016 № 10-92,Ансамбль,POINT (3369290.775 8386927.924),30.266854,30.266854
779,9517,—,Главное здание Государственного оптического ин...,1947-1956,"Кадетская линия, 5, корп. 2, лит. В",объект культурного наследия регионального знач...,Распоряжение КГИОП от 01.07.2019 № 365-р,Памятник,POINT (3372240.630 8387172.627),30.293353,30.293353
780,9605,Два жилых флигеля,Жилой флигель,кон. 18 в.; 1848,"4-я линия В.О., 39, лит. Б",объект культурного наследия регионального знач...,Распоряжение КГИОП от 05.08.2019 № 437-р,Памятник,POINT (3370714.885 8387675.618),30.279647,30.279647


In [126]:
# TODO сохранить слой в географической проекции в формате GeoJSON
rename_gdf.to_crs(4326).to_file('kgiop.geojson', driver='GeoJSON')